# Coset Characters

This is the development notebook for the library that calculates coset characters and partition functions for $\hat{\text{su}}(2)_k$ WZW models and their cosets

## Characters for $\hat{\text{su}}(2)_k$

We know from amazing Di-Francesco that the coset characters of $\hat{\text{su}}(2)_k$ are given by
$$
\chi_{\lambda}^{(k)} = \frac{\Theta^{(k+2)}_{\lambda + 1} - \Theta^{(k+2)}_{-\lambda - 1}}{\Theta^{(2)}_{1} - \Theta^{(2)}_{-1}}.
$$
Therefore in some way we need to calculate these $\Theta$ functions. Luckily he has also done this for us, so in the $q$ expansion we have that
$$
\chi_{\lambda}^{(k)}(q) = q^{\frac{(\lambda+1)^2}{4(k+2)} - \frac{1}{8}} \frac{\sum_{n\in \mathbb{Z}} [\lambda + 1 + 2n(k+2)] q^{n[\lambda + 1 + n(k+2)]}}{\sum_{n \in \mathbb{Z}} (1+4n)q^{n(1+2n)}}
$$
The first sum is some fractional power that contains the central charge term, but the second is not! The second has integer coefficients! The interesting part is calculating this ratio of sums.

We can quickly show that both of the polynomials are of the form
$$
a(q) = \sum_{n\in \mathbb{N}} \hat a(n) q^{\hat f(n)}, 
$$ 
where $\hat f(2n + 1) \coloneqq f(n)$, $\hat f(2n) \coloneqq f(-n)$, and similarly for $\hat a$ where $f(n) = n[\lambda + 1 + n(k+2)]$ is the exponent function over the integers. Not only that, but $\hat f$ is strictly increasing which has some nice properties.

For our numerical evaluation, we effectively have something perfect. We can store everything in a sparse array, and do operations there.

In [ ]:
from sage.all import ZZ, QQ, PowerSeriesRing, latex,O     # Relevant sage objects
from typing import Callable                 # For Typing things


# Claculate the hats
def get_character_int_coefficient_functions(lam:int, k:int):
    funcs = [ 
        lambda n: n*(lam + 1 +n*(k+2)),     # Numerator power
        lambda n: (lam + 1 + 2*n*(k+2)),    # Numerator coefficient
        lambda n: n*(1+2*n),                # Denominator power
        lambda n: 1+4*n                     # Denominator coefficient
    ]

    # Returns positive for even and negative for odd
    def get_hat(F:Callable):
        return lambda n: F([-(n-1)//2,n//2][n&1])
    
    return [get_hat(f) for f in funcs]

# Obtain some cool things
[f,a,g,b] = get_character_int_coefficient_functions(lam = 1, k = 1)


# Let's build some polynomials now
N:int = 1000                                # Number of terms
R = PowerSeriesRing(ZZ,'q', sparse=True)    # Our polynomial ring over the integers
q = R.gen()

numerator_coefficients   = { f(n): a(n) for n in range(1,N) }
denominator_coefficients = { g(n): b(n) for n in range(1,N) }

numerator   = R(numerator_coefficients)
denominator = R(denominator_coefficients)
quotient = numerator*denominator.add_bigoh(N).inverse_of_unit()


In [ ]:
# Also check this out: https://doc.sagemath.org/html/en/thematic_tutorials/lie/integrable.html

2 + 2*q + 6*q^2 + 8*q^3 + 14*q^4 + 20*q^5 + 34*q^6 + 46*q^7 + 70*q^8 + 96*q^9 + 138*q^10 + 186*q^11 + 262*q^12 + 346*q^13 + 472*q^14 + 620*q^15 + 826*q^16 + 1072*q^17 + 1408*q^18 + 1806*q^19 + 2340*q^20 + 2978*q^21 + 3808*q^22 + 4806*q^23 + 6088*q^24 + 7622*q^25 + 9568*q^26 + 11902*q^27 + 14818*q^28 + 18314*q^29 + 22650*q^30 + 27824*q^31 + 34190*q^32 + 41782*q^33 + 51038*q^34 + 62058*q^35 + 75416*q^36 + 91264*q^37 + 110368*q^38 + 132990*q^39 + 160100*q^40 + 192128*q^41 + 230346*q^42 + 275360*q^43 + 328850*q^44 + 391720*q^45 + 466106*q^46 + 553336*q^47 + 656192*q^48 + 776494*q^49 + 917884*q^50 + 1082902*q^51 + 1276208*q^52 + 1501330*q^53 + 1764306*q^54 + 2069870*q^55 + 2425856*q^56 + 2838662*q^57 + 3318312*q^58 + 3873418*q^59 + 4516894*q^60 + 5260096*q^61 + 6119666*q^62 + 7110624*q^63 + 8254208*q^64 + 9570220*q^65 + 11085848*q^66 + 12826866*q^67 + 14828070*q^68 + 17123016*q^69 + 19755940*q^70 + 22770390*q^71 + 26222674*q^72 + 30168840*q^73 + 34680480*q^74 + 39829686*q^75 + 45707024*q^76

In [3]:
# # Define a little printing function for a polynomial up to order n
# from IPython.display import display, Latex

# def show_n(f,n:float=10000, show:bool=True):
#     terms = f.to_dict()
#     monomials = []
#     for power in reversed(terms):
#         if power[0] > n: 
#             break
#         monomials.append(r'%d q^{%d}'%(terms[power], power[0]))

    
#     string = " +".join(monomials)\
#         .replace("+-","-")\
#         .replace(r" q^{0}","")
    
#     if not show:
#         return string
    
#     display(Latex("$"+string+"$"))

# show_n(f)